In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
        
    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i']) 
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)

    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i'])
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    
    if PARAM['atmpro'] == PARAM['atmpro']:
        atmpro = PARAM['atmpro']
    else:
        atmpro = 'mls'
        
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

commitnumber = 523cbb7
molecule = {'h2o': 'atmpro', 'co2': 0.0008}
band = [11]
atmpro = trp
tsfc = 300


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o h2o band9
  atmpro = mls
  band = 9
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 1e-24
  molecule = h2o
  ng_adju = [0]
  ng_refs = [5]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(500, 250)]
  tsfc = 294
  vmax = 3000
  vmin = 1900
  w_diffuse = [(1.66, 1.66, 1.66, 1.66, 1.66)]
  wgt = [(0.4, 0.4, 0.5, 0.55, 0.85)]
o co2 band9
  atmpro = mls
  band = 9
  commitnumber = a06b618
  conc = 0.0004
  dv = 0.001
  klin = 6.5e-24
  molecule = co2
  ng_adju = [0, 0]
  ng_refs = [3, 3]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (50, 250)]
  tsfc = 294
  vmax = 3000
  vmin = 1900
  w_diffuse = [(1.66, 1.66, 1.75), (1.75, 1.6, 1.85)]
  wgt = [(0.7, 0.8, 0.7), (0.8, 0.7, 0.8)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1  0.184203                    1  0.434244
0.000750        2  0.015948                    2  0.039704
0.001052        3  0.016416                    3  0.007066
0.001476        4  0.011189                    4 -0.013961
0.002070        5  0.005500                    5 -0.019924
0.002904        6 -0.000220                    6 -0.023335
0.004074        7 -0.006092                    7 -0.021325
0.005714        8 -0.012193                    8 -0.019899
0.008015        9 -0.018519                    9 -0.019722
0.011243       10 -0.024988                   10 -0.019796
0.015771       11 -0.031599                   11 -0.023074
0.022122       12 -0.038655                   12 -0.028406
0.031031       13 -0.047649                   13 -0.039783
0.043528       14 -0.064059                   14 -0.060478
0.061057       15 -0.075322                   15 -0.072118
0.085645       16 -0.072436                   16 -0.060662
0.120136       17 -0.064109                   17 -0.039368
0.168516       18 -0.052579                   18 -0.022322
0.236378       19 -0.031417                   19 -0.009071
0.331549       20  0.006616                   20  0.010809
0.465100       21  0.083873                   21  0.069383
0.652400       22  0.242337                   22  0.217404
0.915100       23  0.284930                   23  0.272601
1.283650       24  0.203451                   24  0.228411
1.800600       25  0.141161                   25  0.183708
2.525700       26  0.082489                   26  0.115800
3.542800       27  0.047349                   27  0.056302
4.969550       28  0.026714                   28  0.018117
6.970850       29  0.013591                   29  0.008579
9.778100       30  0.005531                   30  0.010014
13.715850      31  0.001134                   31  0.007419
19.239350      32 -0.000710                   32  0.002150
26.987250      33 -0.001112                   33 -0.001473
37.855300      34 -0.000914                   34 -0.001489
53.100050      35 -0.001623                   35 -0.001678
73.887500      36 -0.002293                   36 -0.002373
97.662500      37 -0.002660                   37 -0.002894
121.437500     38 -0.002931                   38 -0.003225
145.212500     39 -0.002905                   39 -0.003217
168.987500     40 -0.003118                   40 -0.003553
192.762500     41 -0.003318                   41 -0.003812
216.537500     42 -0.003500                   42 -0.004059
240.312500     43 -0.003623                   43 -0.004209
264.087500     44 -0.003682                   44 -0.004349
287.862500     45 -0.003668                   45 -0.004343
311.637500     46 -0.003567                   46 -0.004050
335.412500     47 -0.003369                   47 -0.003721
359.187500     48 -0.003076                   48 -0.003398
382.962500     49 -0.002652                   49 -0.003322
406.737500     50 -0.002007                   50 -0.003125
430.512500     51 -0.001146                   51 -0.002518
454.287500     52 -0.000062                   52 -0.001576
478.062500     53  0.001253                   53  0.000233
501.837500     54  0.002801                   54  0.002037
525.612500     55  0.004599                   55  0.004684
549.387500     56  0.006641                   56  0.006902
573.162500     57  0.008954                   57  0.009452
596.937500     58  0.011600                   58  0.011557
620.712500     59  0.014671                   59  0.013627
644.487500     60  0.018383                   60  0.016027
668.262500     61  0.022874                   61  0.019136
692.037500     62  0.027660                   62  0.022787
715.812500     63  0.031241                   63  0.025167
739.587500     64  0.033706                   64  0.027141
763.362500     65  0.032839                   65  0.026255
78

# Fluxes by Layer

CRD                        CLIRAD                \
                       flug      fldg     fnetg      flug          fldg   
pressure    level                                                         
0.000000    1     -4.576204  0.000000 -4.576204 -4.459061  8.551289e-09   
0.000624    2     -4.576209  0.000019 -4.576190 -4.459068  3.949584e-05   
0.000876    3     -4.576212  0.000022 -4.576189 -4.459072  4.432473e-05   
0.001229    4     -4.576215  0.000027 -4.576189 -4.459077  4.928398e-05   
0.001723    5     -4.576220  0.000032 -4.576188 -4.459083  5.455548e-05   
0.002417    6     -4.576226  0.000038 -4.576188 -4.459091  6.078028e-05   
0.003391    7     -4.576234  0.000046 -4.576188 -4.459101  6.856306e-05   
0.004757    8     -4.576245  0.000056 -4.576189 -4.459115  7.898296e-05   
0.006672    9     -4.576259  0.000068 -4.576191 -4.459134  9.303709e-05   
0.009359    10    -4.576280  0.000082 -4.576197 -4.459159  1.121465e-04   
0.013128    11    -4.576308  0.000099 -4.576208 -4.459194  1.383615e-04   
0.018415    12    -4.576347  0.000119 -4.576228 -4.459244  1.738055e-04   
0.025830    13    -4.576405  0.000142 -4.576262 -4.459316  2.213267e-04   
0.036232    14    -4.576491  0.000170 -4.576321 -4.459427  2.828656e-04   
0.050823    15    -4.576636  0.000204 -4.576432 -4.459611  3.621211e-04   
0.071291    16    -4.576886  0.000272 -4.576614 -4.459914  4.905274e-04   
0.100000    17    -4.577290  0.000429 -4.576861 -4.460376  7.466199e-04   
0.140271    18    -4.577913  0.000746 -4.577166 -4.461048  1.230431e-03   
0.196760    19    -4.578856  0.001338 -4.577518 -4.462011  2.043702e-03   
0.275997    20    -4.580244  0.002431 -4.577813 -4.463366  3.313592e-03   
0.387100    21    -4.582165  0.004439 -4.577726 -4.465185  5.275433e-03   
0.543100    22    -4.584287  0.008111 -4.576176 -4.467109  8.481090e-03   
0.761700    23    -4.584270  0.014369 -4.569901 -4.466785  1.378752e-02   
1.068500    24    -4.580860  0.021314 -4.559546 -4.463267  2.017805e-02   
1.498800    25    -4.575632  0.026456 -4.549175 -4.458050  2.660437e-02   
2.102400    26    -4.569152  0.030070 -4.539082 -4.451570  3.326118e-02   
2.949000    27    -4.562606  0.031796 -4.530810 -4.444831  3.813608e-02   
4.136600    28    -4.556383  0.032234 -4.524149 -4.438097  3.932404e-02   
5.802500    29    -4.550540  0.031663 -4.518877 -4.431492  3.629430e-02   
8.139200    30    -4.545167  0.030051 -4.515115 -4.425090  3.226706e-02   
11.417000   31    -4.540375  0.027408 -4.512968 -4.419237  3.030259e-02   
16.014700   32    -4.536294  0.023944 -4.512350 -4.414067  2.917342e-02   
22.464000   33    -4.533054  0.020163 -4.512892 -4.409663  2.641223e-02   
31.510500   34    -4.530731  0.016648 -4.514083 -4.406081  2.125083e-02   
44.200100   35    -4.529045  0.013588 -4.515456 -4.402893  1.582500e-02   
62.000000   36    -4.529134  0.010255 -4.518879 -4.401713  1.110759e-02   
85.775000   37    -4.532548  0.007210 -4.525338 -4.404862  7.572102e-03   
109.550000  38    -4.538631  0.005801 -4.532830 -4.411720  6.279214e-03   
133.325000  39    -4.547480  0.006394 -4.541086 -4.422047  7.522826e-03   
157.100000  40    -4.558538  0.009271 -4.549268 -4.434817  1.123286e-02   
180.875000  41    -4.571921  0.013873 -4.558048 -4.450099  1.650697e-02   
204.650000  42    -4.587806  0.020413 -4.567393 -4.468004  2.367436e-02   
228.425000  43    -4.606638  0.029387 -4.577251 -4.489013  3.324977e-02   
252.200000  44    -4.628814  0.041359 -4.587454 -4.513429  4.581032e-02   
275.975000  45    -4.654833  0.057009 -4.597824 -4.541747  6.187989e-02   
299.750000  46    -4.685286  0.077131 -4.608155 -4.574526  8.242715e-02   
323.525000  47    -4.720846  0.102644 -4.618202 -4.612265  1.087595e-01   
347.300000  48    -4.762267  0.134577 -4.627690 -4.655932  1.419466e-01   
371.075000  49    -4.810214  0.173862 -4.636352 -4.706044  1.824883e-01   
394.850000  50    -4.864370  0.220549 -4.643821 -4.762335  2.294221e-01   
418.625000  51    -4.925123  0.275651 -4.649472 -

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')